# MODEL TRAINING - SET 2

### 1.1 Import Data and Required Packages
#### Importing Pandas, Numpy, Matplotlib, Seaborn and Warnings Library.

In [113]:
#Importing recquired packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

##### Importing Modelling Libraries

In [114]:
#Importing ML Packages
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

from sklearn.multioutput import RegressorChain

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet, Lasso
from sklearn.cross_decomposition import PLSRegression 
from sklearn.svm import SVR                          
from sklearn.tree import DecisionTreeRegressor 
from sklearn.neighbors import KNeighborsRegressor   

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor

from xgboost import XGBRegressor

#Importing DL Packages
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, TensorBoard
from keras.optimizers import Adam
from keras.layers import Dropout, BatchNormalization
import datetime

#### Importing CSV data as Pandas dataframe

In [115]:
#Importing CSV data as Pandas dataframe
df=pd.read_csv('dataset/Extracted_CrudeData.csv')

#Show top 5 rows of the dataframe
df.head()

,Crude Name,StdLiquidDensity (kg/m3),SulfurByWt (%),ConradsonCarbonByWt (%),NitrogenByWt (%),Distillation Mass @ X Pct (C)@ 1 (%) - TBP,Distillation Mass @ X Pct (C)@ 5 (%) - TBP,Distillation Mass @ X Pct (C)@ 10 (%) - TBP,Distillation Mass @ X Pct (C)@ 30 (%) - TBP,Distillation Mass @ X Pct (C)@ 50 (%) - TBP,...,KinematicViscosity (cSt)@ 37.78 (C),KinematicViscosity (cSt)@ 98.89 (C),CetaneNumber,BromineNumber,AnilinePoint (C),FreezePoint (C),PourPoint (C),CloudPoint (C),TotalAcidNumber (mg KOH/g),CtoHRatioByWt
0,Achinsk-2015,926.440713,24.606786,5.827369,0.209874,-17.813183,99.504433,239.896662,398.590562,481.984170,...,57.2992918615153,7.64442822441154,50.267061,NaN,65.639336,108.476408,31.114131,35.355448,0.121042,7.220029
1,Akpo-2014,794.689603,0.070591,0.732083,0.062825,-16.244551,24.679712,74.031996,147.692242,231.447841,...,1.7573028543796,0.833437569080932,48.034358,NaN,10.907278,-32.769863,13.927943,20.861818,0.080432,5.829529
2,AlJurf-2014,874.431106,1.826957,5.241924,0.168953,26.236785,98.856864,138.763063,251.728374,354.117493,...,7.05914911333451,2.02549206169594,53.504942,NaN,40.198516,5.823258,25.601385,31.622843,0.055671,6.654117
3,AlJurf-2015,872.972557,1.840926,5.365742,0.146255,26.408162,96.128866,135.041586,248.512058,351.412080,...,6.77137102006056,2.01032962388146,54.213282,NaN,39.958106,18.377083,27.135976,32.684943,0.028494,6.658939
4,Alba-1994,937.762899,1.322111,5.847739,0.196515,159.292817,225.625562,264.407311,363.170601,470.497570,...,208.950655083803,7.7853062269412,57.443852,NaN,76.496875,1.073170,13.930097,40.746252,1.127930,7.499064


In [116]:
#Shape of dataset
df.shape

(114, 27)

In [117]:
#Renaming columns for better readability
df.rename(columns={
    'Crude Name': 'Crude_Name',
    'StdLiquidDensity (kg/m3)': 'StdLiqDensity_kgm3',
    'SulfurByWt (%)': 'SulfurWt_pct',
    'ConradsonCarbonByWt (%)': 'ConradsonCarbon_wt_pct',
    'NitrogenByWt (%)': 'NitrogenWt_pct',
    
    'Distillation Mass @ X Pct (C)@ 1 (%) - TBP': 'TBP_TempAt_1pct',
    'Distillation Mass @ X Pct (C)@ 5 (%) - TBP': 'TBP_TempAt_5pct',
    'Distillation Mass @ X Pct (C)@ 10 (%) - TBP': 'TBP_TempAt_10pct',
    'Distillation Mass @ X Pct (C)@ 30 (%) - TBP': 'TBP_TempAt_30pct',
    'Distillation Mass @ X Pct (C)@ 50 (%) - TBP': 'TBP_TempAt_50pct',
    'Distillation Mass @ X Pct (C)@ 70 (%) - TBP': 'TBP_TempAt_70pct',
    'Distillation Mass @ X Pct (C)@ 90 (%) - TBP': 'TBP_TempAt_90pct',
    'Distillation Mass @ X Pct (C)@ 95 (%) - TBP': 'TBP_TempAt_95pct',
    'Distillation Mass @ X Pct (C)@ 99 (%) - TBP': 'TBP_TempAt_99pct',

    'AromByWt (%)': 'AromWt_pct',
    'NaphthenesByWt (%)': 'NaphWt_pct',
    'ParaffinsByWt (%)': 'ParaWt_pct',

    'KinematicViscosity (cSt)@ 37.78 (C)': 'KV_37.78c_cSt',
    'KinematicViscosity (cSt)@ 98.89 (C)': 'KV_98.89c_cSt',
    
    "CetaneNumber": 'CetaneNumber',
    "BromineNumber": 'BromineNumber',
    "AnilinePoint (C)": 'AnilinePoint_C',
    "FreezePoint (C)": 'FreezePoint_C',
    "PourPoint (C)": 'PourPoint_C',
    "CloudPoint (C)": 'CloudPoint_C',
    "TotalAcidNumber (mg KOH/g)": 'TotalAcidNumber',
    "CtoHRatioByWt": 'CtoHRatioByWt'
}, inplace=True)
df.head()


,Crude_Name,StdLiqDensity_kgm3,SulfurWt_pct,ConradsonCarbon_wt_pct,NitrogenWt_pct,TBP_TempAt_1pct,TBP_TempAt_5pct,TBP_TempAt_10pct,TBP_TempAt_30pct,TBP_TempAt_50pct,...,KV_37.78c_cSt,KV_98.89c_cSt,CetaneNumber,BromineNumber,AnilinePoint_C,FreezePoint_C,PourPoint_C,CloudPoint_C,TotalAcidNumber,CtoHRatioByWt
0,Achinsk-2015,926.440713,24.606786,5.827369,0.209874,-17.813183,99.504433,239.896662,398.590562,481.984170,...,57.2992918615153,7.64442822441154,50.267061,NaN,65.639336,108.476408,31.114131,35.355448,0.121042,7.220029
1,Akpo-2014,794.689603,0.070591,0.732083,0.062825,-16.244551,24.679712,74.031996,147.692242,231.447841,...,1.7573028543796,0.833437569080932,48.034358,NaN,10.907278,-32.769863,13.927943,20.861818,0.080432,5.829529
2,AlJurf-2014,874.431106,1.826957,5.241924,0.168953,26.236785,98.856864,138.763063,251.728374,354.117493,...,7.05914911333451,2.02549206169594,53.504942,NaN,40.198516,5.823258,25.601385,31.622843,0.055671,6.654117
3,AlJurf-2015,872.972557,1.840926,5.365742,0.146255,26.408162,96.128866,135.041586,248.512058,351.412080,...,6.77137102006056,2.01032962388146,54.213282,NaN,39.958106,18.377083,27.135976,32.684943,0.028494,6.658939
4,Alba-1994,937.762899,1.322111,5.847739,0.196515,159.292817,225.625562,264.407311,363.170601,470.497570,...,208.950655083803,7.7853062269412,57.443852,NaN,76.496875,1.073170,13.930097,40.746252,1.127930,7.499064


#### Duplicate Values, Missing Values & Data Types

In [118]:
#Exploring data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Crude_Name              114 non-null    object 
 1   StdLiqDensity_kgm3      114 non-null    float64
 2   SulfurWt_pct            114 non-null    float64
 3   ConradsonCarbon_wt_pct  114 non-null    float64
 4   NitrogenWt_pct          114 non-null    float64
 5   TBP_TempAt_1pct         114 non-null    float64
 6   TBP_TempAt_5pct         114 non-null    float64
 7   TBP_TempAt_10pct        114 non-null    float64
 8   TBP_TempAt_30pct        114 non-null    float64
 9   TBP_TempAt_50pct        114 non-null    float64
 10  TBP_TempAt_70pct        114 non-null    float64
 11  TBP_TempAt_90pct        114 non-null    float64
 12  TBP_TempAt_95pct        114 non-null    float64
 13  TBP_TempAt_99pct        114 non-null    float64
 14  AromWt_pct              114 non-null    fl

In [119]:
#Changing data types
df["KV_37.78c_cSt"] = pd.to_numeric(df["KV_37.78c_cSt"], errors="coerce")
df["KV_98.89c_cSt"] = pd.to_numeric(df["KV_98.89c_cSt"], errors="coerce")

In [120]:
#Checks for duplicate values
df.duplicated().sum()

np.int64(0)

In [121]:
#Checks for missing values
df.isna().sum()

Crude_Name                  0
StdLiqDensity_kgm3          0
SulfurWt_pct                0
ConradsonCarbon_wt_pct      0
NitrogenWt_pct              0
TBP_TempAt_1pct             0
TBP_TempAt_5pct             0
TBP_TempAt_10pct            0
TBP_TempAt_30pct            0
TBP_TempAt_50pct            0
TBP_TempAt_70pct            0
TBP_TempAt_90pct            0
TBP_TempAt_95pct            0
TBP_TempAt_99pct            0
AromWt_pct                  0
NaphWt_pct                  0
ParaWt_pct                  0
KV_37.78c_cSt               5
KV_98.89c_cSt               2
CetaneNumber                0
BromineNumber             114
AnilinePoint_C              0
FreezePoint_C               0
PourPoint_C                 0
CloudPoint_C                0
TotalAcidNumber             0
CtoHRatioByWt               0
dtype: int64

In [122]:
#Deleting columns with more than 50% missing values
df.drop(columns='BromineNumber', inplace=True)
df.shape

(114, 26)

In [123]:
df.loc[df['Crude_Name'] == 'Cinta-1983', 'KV_37.78c_cSt'] = 14.183399

df = df.dropna(subset=['KV_37.78c_cSt', 'KV_98.89c_cSt'])


#### ML Model Training

In [124]:
#Independent Features
X = df.drop(columns=['Crude_Name', 'AromWt_pct', 'NaphWt_pct', 'ParaWt_pct', 'KV_37.78c_cSt', 'KV_98.89c_cSt', 
                    'CetaneNumber', 'AnilinePoint_C', 'FreezePoint_C', 'PourPoint_C', 'CloudPoint_C', 'TotalAcidNumber', 'CtoHRatioByWt'])
X.head()

,StdLiqDensity_kgm3,SulfurWt_pct,ConradsonCarbon_wt_pct,NitrogenWt_pct,TBP_TempAt_1pct,TBP_TempAt_5pct,TBP_TempAt_10pct,TBP_TempAt_30pct,TBP_TempAt_50pct,TBP_TempAt_70pct,TBP_TempAt_90pct,TBP_TempAt_95pct,TBP_TempAt_99pct
0,926.440713,24.606786,5.827369,0.209874,-17.813183,99.504433,239.896662,398.590562,481.984170,561.500466,665.837149,714.976170,806.217469
1,794.689603,0.070591,0.732083,0.062825,-16.244551,24.679712,74.031996,147.692242,231.447841,311.635864,465.560748,539.770951,687.654783
2,874.431106,1.826957,5.241924,0.168953,26.236785,98.856864,138.763063,251.728374,354.117493,468.885623,655.500149,748.288326,914.614065
3,872.972557,1.840926,5.365742,0.146255,26.408162,96.128866,135.041586,248.512058,351.412080,481.574377,655.699458,736.138972,885.381839
4,937.762899,1.322111,5.847739,0.196515,159.292817,225.625562,264.407311,363.170601,470.497570,595.448493,779.746511,866.765919,1005.918590


In [125]:
#Dependent Features
y = np.log1p(df[['KV_37.78c_cSt', 'KV_98.89c_cSt']]) 

In [126]:
#Split dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")
X_train.shape, X_test.shape, y_train.shape, y_test.shape

Train size: 88, Test size: 22


((88, 13), (22, 13), (88, 2), (22, 2))

In [127]:
#Define categorical and numerical feature columns
num_features = [col for col in X.columns]

#Define transformations for each feature
numeric_transformer = StandardScaler()                    

#Apply scaling to numeric features
preprocessor = ColumnTransformer([("StandardScaler", numeric_transformer, num_features)])


In [128]:
#Machine Learning Algorithms
models = {
    "Linear Regression": Pipeline([
        ('pre', preprocessor),
        ('linreg', MultiOutputRegressor(LinearRegression()))
    ]),
    "Lasso Regression": Pipeline([
        ('pre', preprocessor),
        ('lasso', MultiOutputRegressor(Lasso(alpha=1.0)))
    ]),
    "Ridge Regression": Pipeline([
        ('pre', preprocessor),
        ('ridge', MultiOutputRegressor(Ridge(alpha=1.0)))
    ]),
    "ElasticNet": Pipeline([
        ('pre', preprocessor),
        
        ('elasticnet', MultiOutputRegressor(ElasticNet(alpha=1.0)))
    ]),
    "SVR": Pipeline([
        ('pre', preprocessor),
        
        ('svr', MultiOutputRegressor(SVR(kernel='rbf', C=100, epsilon=0.1)))
    ]),
    "KNN": Pipeline([
        ('pre', preprocessor), 
        ('knn', MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3)))
    ]),
    "Decision Tree Regressor": Pipeline([
        ('pre', preprocessor),  
        ('dt', MultiOutputRegressor(DecisionTreeRegressor(random_state=42)))
    ]),
    "Extra Trees Regressor": Pipeline([
        ('pre', preprocessor),  
        ('et', MultiOutputRegressor(ExtraTreesRegressor(n_estimators=100, random_state=42)))
    ]),
    "Random Forest": Pipeline([
        ('pre', preprocessor),
        ('rf', MultiOutputRegressor(RandomForestRegressor(n_estimators=200, random_state=42)))
    ]),
    "Gradient Boosting": Pipeline([
        ('pre', preprocessor),
        ('gb', MultiOutputRegressor(GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)))
    ]),
    "Adaboost": Pipeline([
        ('pre', preprocessor),
        ('ab', MultiOutputRegressor(AdaBoostRegressor(n_estimators=100, random_state=42)))
    ]),
    "XGBoost": Pipeline([
        ('pre', preprocessor),
        ('xgb', MultiOutputRegressor(XGBRegressor(n_estimators=50, max_depth=3, random_state=42)))
    ]),
    
}

In [129]:
#Train and evaluate all models
print("Training ML Models...")

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {
        'R2': r2_score(y_test, y_pred),
        'MAE': mean_absolute_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
    }
results_df = pd.DataFrame(results).T
print(results_df.sort_values('R2', ascending=False))


Training ML Models...
                               R2       MAE      RMSE
XGBoost                  0.912352  0.289130  0.411552
Linear Regression        0.880096  0.361773  0.511348
Adaboost                 0.867088  0.399807  0.558223
KNN                      0.864505  0.350034  0.530836
Random Forest            0.862764  0.351096  0.534215
Ridge Regression         0.862686  0.379366  0.557568
Extra Trees Regressor    0.857662  0.334423  0.564090
SVR                      0.857315  0.436935  0.618501
Gradient Boosting        0.820831  0.376347  0.639217
Decision Tree Regressor  0.778789  0.454110  0.614479
ElasticNet               0.757199  0.576804  0.700805
Lasso Regression         0.403005  0.830691  0.986404


#### ANN Model Training

In [130]:
#Independent Features
X = df.drop(columns=['Crude_Name', 'AromWt_pct', 'NaphWt_pct', 'ParaWt_pct', 'KV_37.78c_cSt', 'KV_98.89c_cSt', 
                    'CetaneNumber', 'AnilinePoint_C', 'FreezePoint_C', 'PourPoint_C', 'CloudPoint_C', 'TotalAcidNumber', 'CtoHRatioByWt'])
X.head()

,StdLiqDensity_kgm3,SulfurWt_pct,ConradsonCarbon_wt_pct,NitrogenWt_pct,TBP_TempAt_1pct,TBP_TempAt_5pct,TBP_TempAt_10pct,TBP_TempAt_30pct,TBP_TempAt_50pct,TBP_TempAt_70pct,TBP_TempAt_90pct,TBP_TempAt_95pct,TBP_TempAt_99pct
0,926.440713,24.606786,5.827369,0.209874,-17.813183,99.504433,239.896662,398.590562,481.984170,561.500466,665.837149,714.976170,806.217469
1,794.689603,0.070591,0.732083,0.062825,-16.244551,24.679712,74.031996,147.692242,231.447841,311.635864,465.560748,539.770951,687.654783
2,874.431106,1.826957,5.241924,0.168953,26.236785,98.856864,138.763063,251.728374,354.117493,468.885623,655.500149,748.288326,914.614065
3,872.972557,1.840926,5.365742,0.146255,26.408162,96.128866,135.041586,248.512058,351.412080,481.574377,655.699458,736.138972,885.381839
4,937.762899,1.322111,5.847739,0.196515,159.292817,225.625562,264.407311,363.170601,470.497570,595.448493,779.746511,866.765919,1005.918590


In [131]:
#Dependent Features
y = np.log1p(df[['KV_37.78c_cSt', 'KV_98.89c_cSt']])

In [132]:
#Split dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((88, 13), (22, 13), (88, 2), (22, 2))

In [133]:
scaler_X = StandardScaler()
scaler_y = StandardScaler() 

#Fit Train & transform Test
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled  = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled  = scaler_y.transform(y_test)

In [134]:
ann_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.2),
    
    Dense(32, activation='relu'),
    
    Dense(2, activation='linear') 
])
ann_model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 64)             │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,042 (11.88 KB)

 Trainable params: 3,042 (11.88 KB)

 Non-trainable params: 0 (0.00 B)

In [135]:
#Compile the model
ann_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mae'])

In [136]:
#Setup Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [137]:
#Setup Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

In [138]:
print("Training Optimized ANN...")
history = ann_model.fit(X_train_scaled, y_train_scaled, validation_data=(X_test_scaled, y_test_scaled), epochs=1000, batch_size=4,
                  callbacks=[early_stopping, tensorboard_callback])

Training Optimized ANN...
Epoch 1/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - loss: 0.3182 - mae: 0.4292 - val_loss: 0.0913 - val_mae: 0.2195
Epoch 2/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.1879 - mae: 0.3140 - val_loss: 0.1620 - val_mae: 0.3286
Epoch 3/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.1544 - mae: 0.2694 - val_loss: 0.0602 - val_mae: 0.1572
Epoch 4/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.1717 - mae: 0.2986 - val_loss: 0.0539 - val_mae: 0.1679
Epoch 5/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1769 - mae: 0.3004 - val_loss: 0.1870 - val_mae: 0.3191
Epoch 6/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.1792 - mae: 0.2967 - val_loss: 0.2930 - val_mae: 0.3836
Epoch 7/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.2594 - mae: 0.3339 - val_loss: 0.1951 - val_mae: 0.3367
Epoch 8/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1623 - mae: 0.3042 - val_loss: 0.0524 - val_mae: 0.1727
Epoch 9/1000
22/22 ━━━

In [139]:
print("Predicting on Test Set...")
y_pred_scaled = ann_model.predict(X_test_scaled)

#Convert back to real percentages
y_pred_log = scaler_y.inverse_transform(y_pred_scaled)
y_test_log = scaler_y.inverse_transform(y_test_scaled)

y_pred_real = np.expm1(y_pred_log)
y_test_real = np.expm1(y_test_log)

#Calculate Metrics on the Real Data
r2 = r2_score(y_test_real, y_pred_real, multioutput='uniform_average')
mae = mean_absolute_error(y_test_real, y_pred_real)
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_real))

print("ANN PERFORMANCE REPORT")
print(f"R2 Score: {r2:.4f}")
print(f"MAE:      {mae:.4f} %")
print(f"RMSE:     {rmse:.4f} %")
print("-" * 100)

results_df = pd.DataFrame(
    data=np.hstack((y_test_real, y_pred_real)),
    columns=['Actual_KV_37.78c_cSt', 'Pred_KV_37.78c_cSt', 'Actual_KV_98.89c_cSt', 'Pred_KV_98.89c_cSt']
)
print("\nSample Predictions (First 5 Rows):")
display(results_df.head())

Predicting on Test Set...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
ANN PERFORMANCE REPORT
R2 Score: 0.4923
MAE:      111.1610 %
RMSE:     368.4740 %
----------------------------------------------------------------------------------------------------

Sample Predictions (First 5 Rows):


,Actual_KV_37.78c_cSt,Pred_KV_37.78c_cSt,Actual_KV_98.89c_cSt,Pred_KV_98.89c_cSt
0,644.299230,20.650275,254.311966,13.399821
1,5.270543,1.799282,3.154896,1.381028
2,208.950655,7.785306,93.858223,10.582715
3,2313.430743,17.285670,837.148254,20.450060
4,73.341866,9.152543,42.097836,5.926143
